In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)

2.4.1


In [ ]:
####### the multi output selection code
data= []

with open("dino.txt") as f:
  data = f.readlines()

print(data[:10])
oov_token = 'oov'
num_words = 30

['Aachenosaurus\n', 'Aardonyx\n', 'Abdallahsaurus\n', 'Abelisaurus\n', 'Abrictosaurus\n', 'Abrosaurus\n', 'Abydosaurus\n', 'Acanthopholis\n', 'Achelousaurus\n', 'Acheroraptor\n']


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=num_words,char_level=True)
tokenizer.fit_on_texts(data)
word_index = tokenizer.word_index
print(word_index)

{'a': 1, 's': 2, 'u': 3, 'o': 4, 'r': 5, '\n': 6, 'n': 7, 'i': 8, 'e': 9, 't': 10, 'l': 11, 'p': 12, 'h': 13, 'c': 14, 'g': 15, 'd': 16, 'm': 17, 'y': 18, 'b': 19, 'k': 20, 'v': 21, 'x': 22, 'z': 23, 'j': 24, 'w': 25, 'f': 26, 'q': 27}


In [ ]:
data_seq = tokenizer.texts_to_sequences(data)
print(data[0])
print(data_seq[0])

Aachenosaurus

[1, 1, 14, 13, 9, 7, 4, 2, 1, 3, 5, 3, 2, 6]


In [ ]:
inp_X = []
inp_Y = []

for seq in data_seq :
  for i in range(len(seq)-1):
    inp_X.append(seq[:i+1])
    inp_Y.append(seq[1:i+2])
#print(inp_X)
#print(inp_Y)

print(len(inp_X),len(inp_Y))
print(inp_X[1],inp_Y[1])

18373 18373
[1, 1] [1, 14]


In [ ]:
maxlen = max([len(seq) for seq in inp_X])
print(maxlen)

26


In [ ]:
train_X = pad_sequences(inp_X,maxlen)
train_Y = pad_sequences(inp_Y,maxlen)

print(train_X.shape)
print(train_Y.shape)

(18373, 26)
(18373, 26)


In [ ]:
input = layers.Input(shape=(None,))
embedding_out = layers.Embedding(input_dim=num_words,output_dim=5)(input)
rnn_out,state_h,state_c = layers.LSTM(50,return_sequences=True,return_state=True)(embedding_out)
output = layers.Dense(30,activation='softmax')(rnn_out)


In [ ]:
model = tf.keras.Model(input,output)
model.summary()
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 5)           150       
_________________________________________________________________
lstm_2 (LSTM)                [(None, None, 50), (None, 11200     
_________________________________________________________________
dense_2 (Dense)              (None, None, 30)          1530      
Total params: 12,880
Trainable params: 12,880
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_X[:10000],train_Y[:10000],epochs=20)

Epoch 1/20
313/313 [==============================] - 3s 5ms/step - loss: 1.7556 - accuracy: 0.7264
Epoch 2/20
313/313 [==============================] - 2s 5ms/step - loss: 0.7459 - accuracy: 0.7776
Epoch 3/20
313/313 [==============================] - 2s 5ms/step - loss: 0.6705 - accuracy: 0.7919
Epoch 4/20
313/313 [==============================] - 2s 5ms/step - loss: 0.6214 - accuracy: 0.8113
Epoch 5/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5977 - accuracy: 0.8189
Epoch 6/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5704 - accuracy: 0.8244
Epoch 7/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5621 - accuracy: 0.8264
Epoch 8/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5473 - accuracy: 0.8312
Epoch 9/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5427 - accuracy: 0.8321
Epoch 10/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5274 - accuracy: 0.8355

In [ ]:
test_X = np.array(train_X[25])
print(test_X.shape)
test_X = np.reshape(test_X,[1,-1])
print(test_X.shape)
res = model.predict(test_X)
print(res.shape)

(26,)
(1, 26)
(1, 26, 30)


In [ ]:
sample = model.layers[1](test_X)
print(sample.shape)
sample_2,state_h,state_c = model.layers[2](sample)
sample_3 = model.layers[3](sample_2)
print(sample_2.shape)
print(state_h.shape,state_c.shape)
print(sample_3.shape)

(1, 26, 5)
(1, 26, 50)
(1, 50) (1, 50)
(1, 26, 30)


In [ ]:
rev_sample = tokenizer.sequences_to_texts(test_X)
print(rev_sample)
print(test_X)

print(sample_3)
print(np.argmax(sample_3))

['a b d a l']
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 19 16
   1 11]]
tf.Tensor(
[[3.5107674e-12 8.2286970e-05 3.6094137e-03 2.6167643e-05 2.9124146e-02
  6.4744152e-02 1.1015487e-06 2.8948888e-02 6.0967356e-03 3.7313244e-04
  7.0089186e-03 8.0798131e-01 5.2713098e-05 4.0368942e-04 5.5132178e-03
  3.2131058e-05 1.9257027e-04 8.6125219e-03 5.3172791e-04 1.2096987e-03
  3.5049975e-02 1.4663447e-04 6.2865379e-06 8.1919425e-05 2.4894337e-06
  4.8081401e-05 1.2012369e-04 1.0039261e-07 3.6516228e-12 5.6560468e-12]], shape=(1, 30), dtype=float32)
11


In [ ]:
seed_text = ["H"]

i = 0
next_char = seed_text

test_seq = tokenizer.texts_to_sequences(seed_text)

print(test_seq)
test_seq = np.array(test_seq)
#test_seq = np.reshape(test_seq,[1,-1)
test_out1 = model.layers[1](test_seq)
print(test_out1.shape)
test_out2,state_h,state_c = model.layers[2](test_out1)
test_out3 = model.layers[3](test_out2)
print(test_out3.shape,state_h.shape,state_c.shape)

predicted = np.argmax(test_out3)

test_seq2 = tokenizer.sequences_to_texts([[predicted]])
print(test_seq2)

[[13]]
(1, 1, 5)
(1, 30) (1, 50) (1, 50)
['p']


In [ ]:

i = 0
seed_text = ["megold"]
result = seed_text[0]
next_char = seed_text
while next_char[0] != '\n':

  test_seq = tokenizer.texts_to_sequences(next_char)
  test_seq = np.array(test_seq)
  embedding_out =model.layers[1](test_seq)
  if (i == 0):
    state_h,state_c = model.layers[2].get_initial_state(embedding_out)
  
  lstm_out,state_h,state_c = model.layers[2](embedding_out,initial_state = [state_h,state_c])
  dense_out = model.layers[3](lstm_out[:,-1,:])

  char_id = np.argmax(dense_out)
  next_char = tokenizer.sequences_to_texts([[char_id]])
  #print(next_char)
  result += next_char[0]

  print(result)
  i = i+1
  #if(i >40):
  #  break

  if(next_char[0] =='\n'):
    break

megoldo
megoldos
megoldosa
megoldosau
megoldosaur
megoldosauru
megoldosaurus
megoldosaurus

